In [1]:
import neo4j
import pandas as pd
import geopy.distance

### Search for good osmid pairs

## Neo4j Import

In [4]:
NEO4J_URI = "bolt://localhost:7690"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "graph-routing"
NEO4J_DATABASE = "neo4j"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), database=NEO4J_DATABASE)

In [5]:
clear_data_query = """
    MATCH (n) 
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS OF 100000 ROWS
"""

clear_indexes_and_constrains = "CALL apoc.schema.assert({}, {}, true) YIELD label, key RETURN *"

node_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (i:Intersection) REQUIRE i.osmid IS UNIQUE"

rel_index_query = "CREATE INDEX IF NOT EXISTS FOR ()-[r:ROAD_SEGMENT]-() ON r.osmid"

# TODO: remember ro copy files to import folder in neo4j and change filenames in query

nodes_csv_load = """
    LOAD CSV WITH HEADERS FROM "file:///krakow_big/nodes_neo4j.csv" AS row
    CALL {
        WITH row
        MERGE (i:Intersection {osmid: toInteger(row.osmid)})
        SET i.latitude = toFloat(row.y), 
            i.longitude = toFloat(row.x)
    } IN TRANSACTIONS OF 100000 ROWS
    RETURN COUNT(*) as total
    """
    
relationships_csv_load = """
    LOAD CSV WITH HEADERS FROM "file:///krakow_big/relationships_neo4j.csv" AS row
    CALL {
        WITH row
        MATCH (from:Intersection {osmid: toInteger(row.source)})
        MATCH (to:Intersection {osmid: toInteger(row.target)})
        MERGE (from)-[r:ROAD_SEGMENT {osmid: toInteger(row.osmid)}]->(to)
        SET r.length = toFloat(row.length)
    } IN TRANSACTIONS OF 100000 ROWS
    RETURN COUNT(*) AS total
    """


In [6]:
def print_result(results):
    result = [dict(i) for i in results]
    print(result)
    
    result_all = results.consume()
    print(f"result_consumed_after: {result_all.result_consumed_after} ms")
    print(f"result_available_after: {result_all.result_available_after} ms\n")

In [7]:
def delete_database(tx):
    results = tx.run(clear_data_query)
    print_result(results)

def clear_indexes(tx):
    results = tx.run(clear_indexes_and_constrains)
    print_result(results)

In [8]:
def create_constraints(tx):
    results = tx.run(node_constraint_query)
    print_result(results)

    results = tx.run(rel_index_query)
    print_result(results)

### Clear database

In [10]:
with driver.session() as session:
    session.execute_write(delete_database)
    
with driver.session() as session:
    session.execute_write(clear_indexes)

Transaction failed and will be retried in 0.9289004472081769s (The allocation of an extra 96.0 MiB would use more than the limit 3.5 GiB. Currently using 3.4 GiB. dbms.memory.transaction.total.max threshold reached)
Transaction failed and will be retried in 1.7834476141724305s (The allocation of an extra 96.0 MiB would use more than the limit 3.5 GiB. Currently using 3.4 GiB. dbms.memory.transaction.total.max threshold reached)


DatabaseError: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: Java heap space}

### Create constraints

In [12]:
with driver.session() as session:
    session.execute_write(create_constraints)

[]
result_consumed_after: 0 ms
result_available_after: 46 ms

[]
result_consumed_after: 0 ms
result_available_after: 16 ms



### Load data

In [13]:
with driver.session() as session:
    results = session.run(nodes_csv_load)
    print_result(results)

[{'total': 31096557}]
result_consumed_after: 1666588 ms
result_available_after: 261 ms



In [9]:
with driver.session() as session:
    results = session.run(relationships_csv_load)
    print_result(results)

[{'total': 65135420}]
result_consumed_after: 3179641 ms
result_available_after: 1013 ms



### Search queries

```
EXPLAIN PROFILE
```

In [7]:
def dijkstra_query(tx, osmid_1, osmid_2):
    dijkstra_query = f"""
        MATCH (source:Intersection {{osmid: {osmid_1}}}) 
        MATCH (target:Intersection {{osmid: {osmid_2}}})
        CALL apoc.algo.dijkstra(source, target, "", "length")
        YIELD path, weight
        RETURN path, weight
        """
    results = tx.run(dijkstra_query)
    print_result(results)

In [8]:
def astar_query(tx, osmid_1, osmid_2):
    astar_query = f"""
        MATCH (source:Intersection {{osmid: {osmid_1}}}) 
        MATCH (target:Intersection {{osmid: {osmid_2}}})
        CALL apoc.algo.aStar(source, target, "", "length", "latitude", "longitude")
        YIELD path, weight
        RETURN path, weight
        """
    results = tx.run(astar_query)
    print_result(results)

In [35]:
gdf_nodes = pd.read_csv("./data/krakow_nodes_neo4j.csv")

In [216]:
# two random osmid
start_and_finish = gdf_nodes.sample(2)

osmid_1 = start_and_finish.iloc[0]["osmid"]
osmid_2 = start_and_finish.iloc[1]["osmid"]

coords_1 = start_and_finish.iloc[0]["y"], start_and_finish.iloc[0]["x"]
coords_2 = start_and_finish.iloc[1]["y"], start_and_finish.iloc[1]["x"]

print(f"osmid_1: {osmid_1}, osmid_2: {osmid_2}, distance: {geopy.distance.distance(coords_1, coords_2).km} km")

osmid_1: 10298662254.0, osmid_2: 2139306598.0, distance: 24.768543728456766 km


In [19]:
# zakamycze, betel
# node_nr_1 = 356926768
# node_nr_2 = 2104495834

# chrzanów, brzesko
node_nr_1 = 552992131
node_nr_2 = 6191458019

In [22]:
with driver.session() as session:
    session.execute_write(dijkstra_query, node_nr_1, node_nr_2)

In [21]:
with driver.session() as session:
    session.execute_write(astar_query, node_nr_1, node_nr_2)

[]
result_consumed_after: 13237 ms
result_available_after: 19 ms



### Add addresses - TODO

In [ ]:
# We'll use apoc.load.json to import a JSON file of address data

address_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Address) REQUIRE a.id IS UNIQUE"

add_addresses_query = """
CALL apoc.periodic.iterate(
  'CALL apoc.load.json("/home/pcend/Piotr/kod/lisboa-routing/data/lisboa.geojson") YIELD value',
  'MERGE (a:Address {id: value.properties.id})
SET a.location = 
  point(
      {latitude: value.geometry.coordinates[1], longitude: value.geometry.coordinates[0]}),
    a.full_address = value.properties.number + " " + value.properties.street + " " + value.properties.city + ", CA " + value.properties.postcode

SET a += value.properties',
  {batchSize:10000, parallel:true})
"""

# Next, connect each address to the road network at the nearest intersection

near_intersection_query = """
CALL apoc.periodic.iterate(
  'MATCH (p:Address) WHERE NOT EXISTS ((p)-[:NEAREST_INTERSECTION]->(:Intersection)) RETURN p',
  'CALL {
  WITH p
  MATCH (i:Intersection)
  USING INDEX i:Intersection(location)
  WHERE point.distance(i.location, p.location) < 200

  WITH i
  ORDER BY point.distance(p.location, i.location) ASC 
  LIMIT 1
  RETURN i
}
WITH p, i

MERGE (p)-[r:NEAREST_INTERSECTION]->(i)
SET r.length = point.distance(p.location, i.location)
RETURN COUNT(p)',
  {batchSize:1000, parallel:false})
"""

# Create a full text index to enable search in our web app

full_text_query = "CREATE FULLTEXT INDEX search_index IF NOT EXISTS FOR (p:PointOfInterest|Address) ON EACH [p.name, p.full_address]"

In [ ]:
def enrich_addresses(tx):
    results = tx.run(add_addresses_query)        
    results = tx.run(near_intersection_query)


In [ ]:
# with driver.session() as session:
#     session.execute_write(enrich_addresses)

In [ ]:
# with driver.session() as session:
#     results = session.execute_write(lambda tx: tx.run(full_text_query))


### Query with address

```
MATCH (a:Address)-[:NEAREST_INTERSECTION]->(source:Intersection)
WHERE a.full_address CONTAINS "410 E 5TH AVE SAN MATEO, CA"
MATCH 
  (poi:Address)-[:NEAREST_INTERSECTION]->(dest:Intersection) 
WHERE poi.full_address CONTAINS "111 5TH AVE"
CALL apoc.algo.dijkstra(source, dest, "ROAD_SEGMENT", "length") 
YIELD weight, path
RETURN **
```


```
CALL db.index.fulltext.queryNodes("search_index", $searchString) 
YIELD node, score
RETURN coalesce(node.name, node.full_address) AS value, score, labels(node)[0] AS label, node.id AS id
ORDER BY score DESC LIMIT 25
```

```
MATCH (to {id: $dest})-[:NEAREST_INTERSECTION]->(source:Intersection) 
MATCH (from {id: $source})-[:NEAREST_INTERSECTION]->(target:Intersection)
CALL apoc.algo.dijkstra(source, target, 'ROAD_SEGMENT', 'length')
YIELD path, weight
RETURN [n in nodes(path) | [n.location.latitude, n.location.longitude]] AS route
```
